In [1]:
!pip install torch torchvision matplotlib numpy scikit-learn


In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from PIL import Image


In [3]:
# Define dataset paths
train_data_dir = "Dataset_new (1)/VARICOSE_VEINS_own/train/"
test_data_dir = "Dataset_new (1)/VARICOSE_VEINS_own/test/"
valid_data_dir = "Dataset_new (1)/VARICOSE_VEINS_own/valid/"

# Hyperparameters
img_height, img_width = 224, 224
batch_size = 16
num_classes = 2
learning_rate = 0.001
num_epochs = 40#30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
print(f"Train dataset size: {len(train_data_dir)}")
print(f"Validation dataset size: {len(test_data_dir)}")
print(f"Test dataset size: {len(valid_data_dir)}")

Train dataset size: 41
Validation dataset size: 40
Test dataset size: 41


In [5]:
# Define transformations
train_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=test_transforms)
valid_dataset = datasets.ImageFolder(root=valid_data_dir, transform=test_transforms)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Class names
class_names = train_dataset.classes
print("Class Names:", class_names)


Class Names: ['normal', 'varicose']


In [6]:
# Load pretrained ResNeXt-101 model
model = models.resnext101_32x8d(pretrained=True)

# Modify the classifier
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, num_classes),
    nn.Softmax(dim=1)
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(model)

C:\Users\Vinod\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Vinod\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [ ]:
train_loss_values = []
valid_loss_values = []
train_acc_values = []
valid_acc_values = []

for epoch in range(num_epochs):
    model.train()
    train_loss, correct_train = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()

    train_loss /= len(train_loader)
    train_acc = correct_train / len(train_dataset)
    train_loss_values.append(train_loss)
    train_acc_values.append(train_acc)

    model.eval()
    valid_loss, correct_valid = 0, 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_valid += (predicted == labels).sum().item()

    valid_loss /= len(valid_loader)
    valid_acc = correct_valid / len(valid_dataset)
    valid_loss_values.append(valid_loss)
    valid_acc_values.append(valid_acc)

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
          f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

# Save the trained model
torch.save(model.state_dict(), "varicose_resnext101.pth")
print("Model saved successfully!")


Epoch 1/40 - Train Loss: 0.5433, Train Acc: 0.7476 - Valid Loss: 0.8203, Valid Acc: 0.5000
Epoch 2/40 - Train Loss: 0.5655, Train Acc: 0.7619 - Valid Loss: 0.6458, Valid Acc: 0.6739
Epoch 3/40 - Train Loss: 0.5185, Train Acc: 0.7667 - Valid Loss: 0.5490, Valid Acc: 0.7609
Epoch 4/40 - Train Loss: 0.4893, Train Acc: 0.8000 - Valid Loss: 0.5570, Valid Acc: 0.7609
